# Analise Exploratória de desempenho - Encontrando indicadores e filtrando Fundos

In [1]:
from dateutil import rrule
from datetime import datetime, timedelta
import calendar
import pandas as pd
import datetime as dt
import os
import re 
import numpy as np
from tqdm import tqdm

In [ ]:
# Formulas recorrentes referentes à cada coluna em nossa tabela

def cumulative_returns(df):
    if df.empty:
        return 0
    else:
        return (df.iloc[-1]/df.iloc[0])-1

def daily_returns(df):
    dr = (df/df.shift(1)) - 1
    dr = dr[1:]
    return dr

def max_daily_return(df):
    print(df)
    df = daily_returns(df)
    max_value = df.max()
    index = df.index[df == max_value]
    return max_value, index

def average_daily_returns(df):
    dr = daily_returns(df)
    return dr.mean()

def average_monthly_return(df):
    first = df['DT_COMPTC'].iloc[0]
    first = first.replace(day=1)
    last = df['DT_COMPTC'].iloc[-1]
    last = last.replace(day=1)
    month_return_list = []
    for date in rrule.rrule(rrule.MONTHLY, dtstart=first, until=last):
        mask = df['DT_COMPTC'].dt.year == date.year
        filtered_date = quotas.loc[mask]
        mask = df['DT_COMPTC'].dt.month == date.month
        filtered_date = filtered_date.loc[mask]
        month_return = cumulative_returns(filtered_date['VL_QUOTA'])
        
        month_return_list.append([date, month_return])

        monthly_df = pd.DataFrame(month_return_list, columns=['month', 'return'])
    
    return monthly_df['return'].mean()

def annualized_return_function(df):
    first = df['DT_COMPTC'].iloc[0]
    first = first.replace(day=1)
    last = df['DT_COMPTC'].iloc[-1]
    last = last.replace(day=1)
    ann_return_list = []
    for date in rrule.rrule(rrule.YEARLY, dtstart=first, until=last):
        mask = df['DT_COMPTC'].dt.year == date.year
        filtered_date = quotas.loc[mask]
        ann_return = cumulative_returns(filtered_date['VL_QUOTA'])
        
        ann_return_list.append([date, ann_return])

        ann_df = pd.DataFrame(ann_return_list, columns=['year', 'return'])
    
    return ann_df['return'].mean()

def volatility(df):
    dr = daily_returns(df)
    return dr.std()

def Sharpe_Ratio(df,rf=0):
    samples_per_year = 252
    mean_avg_daily_rets = (average_daily_returns(df) - rf).mean()
    vol = volatility(df)
    sharpe_ratio = np.sqrt(samples_per_year)*(mean_avg_daily_rets/vol)
    return sharpe_ratio

def BTR(fi_cumulative_returns, fi_returns,bench_returns,rf=0):
    covariance = np.cov(fi_returns,bench_returns)[0][1]
    variance = np.var(bench_returns)
    beta = covariance/variance
    Treynor_Ratio = (fi_cumulative_returns - rf)/beta
    return Treynor_Ratio

In [ ]:
# Criando e salvando tabela de desempenho

unique_fi = pd.read_csv('unique_fi.csv', index_col='CNPJ_FUNDO')
cnpj_files = [c for c in os.listdir('CNPJ desempenho/')]

cnpj_list = unique_fi.index.tolist()

try:
    calculated_cnpj = pd.read_csv('desempenho.csv')
    calculated_cnpj_list = calculated_cnpj.CNPJ_FUNDO.tolist()

    cnpj_list = [x for x in cnpj_list if x not in calculated_cnpj_list]
except:
    pass

# gerando planilha de retorno diário CDI
cdi = pd.read_csv('cdi.csv')
cdi.drop(cdi.columns.difference(['data', 'cdi_return']), 1, inplace=True)
cdi.columns = ['DT_COMPTC', 'cdi_return']
cdi['DT_COMPTC'] = pd.to_datetime(cdi['DT_COMPTC'])

# gerando planilha de retorno diário IBOV
ibov = pd.read_csv('ibov.csv')
ibov['returns'] = daily_returns(ibov['Adj Close'])
ibov.drop(ibov.columns.difference(['Date', 'returns']), 1, inplace=True)
ibov.columns = ['DT_COMPTC', 'ibov_return']
ibov['DT_COMPTC'] = pd.to_datetime(ibov['DT_COMPTC'])

# gerando planilha de retorno diário S&p
snp = pd.read_csv('snp.csv')
snp['returns'] = daily_returns(snp['snp_brl_close'])
snp.drop(snp.columns.difference(['Date', 'returns']), 1, inplace=True)
snp.columns = ['DT_COMPTC', 'snp_return']
snp['DT_COMPTC'] = pd.to_datetime(snp['DT_COMPTC'])

desempenho = []

month_1 = pd.Timedelta(value=30, unit='days')

pbar = tqdm(cnpj_list)
for cnpj in pbar:
    pbar.set_description("Processing %s" % cnpj)
    filename = f'{re.sub("[^0-9]", "", cnpj)}.csv'
    quotas = pd.read_csv(f'CNPJ desempenho/{filename}')

    # convertendo datas em string para datetime
    quotas['DT_COMPTC'] = pd.to_datetime(quotas['DT_COMPTC'])


    # Removendo cotas zeradas (declaradas antes do fundo estar em funcionamento ou depois de canceladas)
    quotas = quotas[quotas.VL_QUOTA != 0]

    # gerando retornos
    quotas['RETORNO_DIARIO'] = daily_returns(quotas['VL_QUOTA'])



    # left join com cdi
    quotas = pd.merge(quotas, cdi, on='DT_COMPTC', how='left')

    # left join com ibov
    quotas = pd.merge(quotas, ibov, on='DT_COMPTC', how='left')

    # left join com S&P
    quotas = pd.merge(quotas, snp, on='DT_COMPTC', how='left')

    # Keep only collums of interest
    quotas.drop(quotas.columns.difference(['DT_COMPTC', 'VL_QUOTA', 'RETORNO_DIARIO', 'cdi_return', 'ibov_return', 'snp_return']), 1, inplace=True)

    # filling days with no return at all (Null values)
    quotas = quotas.fillna(value=0)
    
    if len(quotas) > 1:

        # ytd    
        mask = quotas['DT_COMPTC'].dt.year == int(2021)
        filtered_date = quotas[mask]
        if filtered_date.empty:
            retorno_ytd = None
        else: 
            retorno_ytd = cumulative_returns(filtered_date['VL_QUOTA'])

        # 2020    
        mask = quotas['DT_COMPTC'].dt.year == int(2020)
        filtered_date = quotas[mask]

        if filtered_date.empty:
            retorno_2020 = None
        else:    
            retorno_2020 = cumulative_returns(filtered_date['VL_QUOTA'])

        # 1 mês
        mask = quotas['DT_COMPTC'] >= (pd.to_datetime(f"{quotas['DT_COMPTC'].iloc[-1].year}-{quotas['DT_COMPTC'].iloc[-1].month}-{quotas['DT_COMPTC'].iloc[-1].day}")) - month_1
        filtered_date = quotas.loc[mask]
        inverse = quotas.loc[quotas.index.difference(filtered_date.index)]
        if inverse.empty:
            retorno_1_mes = None
        else:
            retorno_1_mes = cumulative_returns(filtered_date['VL_QUOTA'])

        # 3 meses
        mask = quotas['DT_COMPTC'] >= (pd.to_datetime(f"{quotas['DT_COMPTC'].iloc[-1].year}-{quotas['DT_COMPTC'].iloc[-1].month}-{quotas['DT_COMPTC'].iloc[-1].day}")) - 3*month_1
        filtered_date = quotas.loc[mask]
        inverse = quotas.loc[quotas.index.difference(filtered_date.index)]
        if inverse.empty:
            retorno_3_meses = None
        else:
            retorno_3_meses = cumulative_returns(filtered_date['VL_QUOTA'])

        # 1 ano        
        # caso ano seja bissexto
        if quotas['DT_COMPTC'].iloc[-1].month == 2 and quotas['DT_COMPTC'].iloc[-1].day == 29:
            mask = quotas['DT_COMPTC'] >= pd.to_datetime(f"{quotas['DT_COMPTC'].iloc[-1].year-1}-{quotas['DT_COMPTC'].iloc[-1].month}-{quotas['DT_COMPTC'].iloc[-1].day-1}")
        else:
            mask = quotas['DT_COMPTC'] >= pd.to_datetime(f"{quotas['DT_COMPTC'].iloc[-1].year-1}-{quotas['DT_COMPTC'].iloc[-1].month}-{quotas['DT_COMPTC'].iloc[-1].day}")

        filtered_date = quotas.loc[mask]
        inverse = quotas.loc[quotas.index.difference(filtered_date.index)]

        if inverse.empty:
            retorno_1_ano = None
            traynor_cdi_1ano = None
            traynor_ibov_1ano = None
            traynor_snp_1ano = None
        else:   
            retorno_1_ano = cumulative_returns(filtered_date['VL_QUOTA'])
            traynor_cdi_1ano = BTR(retorno_1_ano, filtered_date['RETORNO_DIARIO'],filtered_date['cdi_return'])
            traynor_ibov_1ano = BTR(retorno_1_ano, filtered_date['RETORNO_DIARIO'],filtered_date['ibov_return'])
            traynor_snp_1ano = BTR(retorno_1_ano, filtered_date['RETORNO_DIARIO'],filtered_date['snp_return'])


        # 3 anos
        # caso ano seja bissexto
        if quotas['DT_COMPTC'].iloc[-1].month == 2 and quotas['DT_COMPTC'].iloc[-1].day == 29:
            mask = quotas['DT_COMPTC'] >= pd.to_datetime(f"{quotas['DT_COMPTC'].iloc[-1].year-3}-{quotas['DT_COMPTC'].iloc[-1].month}-{quotas['DT_COMPTC'].iloc[-1].day-1}")
        else:
            mask = quotas['DT_COMPTC'] >= pd.to_datetime(f"{quotas['DT_COMPTC'].iloc[-1].year-3}-{quotas['DT_COMPTC'].iloc[-1].month}-{quotas['DT_COMPTC'].iloc[-1].day}")

        filtered_date = quotas.loc[mask]
        inverse = quotas.loc[quotas.index.difference(filtered_date.index)]
        if inverse.empty:
            retorno_3_anos = None
            traynor_cdi_3ano = None
            traynor_ibov_3ano = None
            traynor_snp_3ano = None
        else:
            retorno_3_anos = cumulative_returns(filtered_date['VL_QUOTA'])
            traynor_cdi_3ano = BTR(retorno_3_anos, filtered_date['RETORNO_DIARIO'],filtered_date['cdi_return'])
            traynor_ibov_3ano = BTR(retorno_3_anos, filtered_date['RETORNO_DIARIO'],filtered_date['ibov_return'])
            traynor_snp_3ano = BTR(retorno_3_anos, filtered_date['RETORNO_DIARIO'],filtered_date['snp_return'])      

        # 5 anos
        # caso ano seja bissexto
        if quotas['DT_COMPTC'].iloc[-1].month == 2 and quotas['DT_COMPTC'].iloc[-1].day == 29:
            mask = quotas['DT_COMPTC'] >= pd.to_datetime(f"{quotas['DT_COMPTC'].iloc[-1].year-5}-{quotas['DT_COMPTC'].iloc[-1].month}-{quotas['DT_COMPTC'].iloc[-1].day-1}")
        else:
            mask = quotas['DT_COMPTC'] >= pd.to_datetime(f"{quotas['DT_COMPTC'].iloc[-1].year-5}-{quotas['DT_COMPTC'].iloc[-1].month}-{quotas['DT_COMPTC'].iloc[-1].day}")

        
        filtered_date = quotas.loc[mask]
        inverse = quotas.loc[quotas.index.difference(filtered_date.index)]
        if inverse.empty:
            retorno_5_anos = None
            traynor_cdi_5ano = None
            traynor_ibov_5ano = None
            traynor_snp_5ano = None  
        else:    
            retorno_5_anos = cumulative_returns(filtered_date['VL_QUOTA'])
            traynor_cdi_5ano = BTR(retorno_5_anos, filtered_date['RETORNO_DIARIO'],filtered_date['cdi_return'])
            traynor_ibov_5ano = BTR(retorno_5_anos, filtered_date['RETORNO_DIARIO'],filtered_date['ibov_return'])
            traynor_snp_5ano = BTR(retorno_5_anos, filtered_date['RETORNO_DIARIO'],filtered_date['snp_return'])   


        # 10 anos
        # caso ano seja bissexto
        if quotas['DT_COMPTC'].iloc[-1].month == 2 and quotas['DT_COMPTC'].iloc[-1].day == 29:
            mask = quotas['DT_COMPTC'] >= pd.to_datetime(f"{quotas['DT_COMPTC'].iloc[-1].year-10}-{quotas['DT_COMPTC'].iloc[-1].month}-{quotas['DT_COMPTC'].iloc[-1].day-1}")
        else:
            mask = quotas['DT_COMPTC'] >= pd.to_datetime(f"{quotas['DT_COMPTC'].iloc[-1].year-10}-{quotas['DT_COMPTC'].iloc[-1].month}-{quotas['DT_COMPTC'].iloc[-1].day}")

            
        filtered_date = quotas.loc[mask]
        inverse = quotas.loc[quotas.index.difference(filtered_date.index)]
        if inverse.empty:
            retorno_10_anos = None
            traynor_cdi_10ano = None
            traynor_ibov_10ano = None
            traynor_snp_10ano = None        
        else:    
            retorno_10_anos = cumulative_returns(filtered_date['VL_QUOTA'])
            traynor_cdi_10ano = BTR(retorno_10_anos, filtered_date['RETORNO_DIARIO'],filtered_date['cdi_return'])
            traynor_ibov_10ano = BTR(retorno_10_anos, filtered_date['RETORNO_DIARIO'],filtered_date['ibov_return'])
            traynor_snp_10ano = BTR(retorno_10_anos, filtered_date['RETORNO_DIARIO'],filtered_date['snp_return'])    

        # retorno acumulado e redimento diário médio
        fi_cumulative_returns = cumulative_returns(quotas['VL_QUOTA'])

        fi_average_daily_returns = average_daily_returns(quotas['VL_QUOTA'])

        # annualized sharpe
        annualized_sharpe = Sharpe_Ratio(quotas['VL_QUOTA'])


        desempenho.append([cnpj, fi_cumulative_returns, fi_average_daily_returns, annualized_sharpe,
                           retorno_ytd, retorno_2020, retorno_1_mes, retorno_3_meses,
                           retorno_1_ano, traynor_cdi_1ano, traynor_ibov_1ano, traynor_snp_1ano,
                           retorno_3_anos, traynor_cdi_3ano, traynor_ibov_3ano, traynor_snp_3ano,
                           retorno_5_anos, traynor_cdi_5ano, traynor_ibov_5ano, traynor_snp_5ano,
                           retorno_10_anos, traynor_cdi_10ano, traynor_ibov_10ano, traynor_snp_10ano])


desempenho_df = pd.DataFrame(desempenho,columns = ['CNPJ_FUNDO', 'cumulative_returns', 'average_daily_returns', 'annualized_sharpe',
                                                   'retorno_ytd', 'retorno_2020', 'retorno_1_mes', 'retorno_3_meses',
                                                   'retorno_1_ano', 'traynor_cdi_1ano', 'traynor_ibov_1ano', 'traynor_snp_1ano',
                                                   'retorno_3_anos', 'traynor_cdi_3ano', 'traynor_ibov_3ano', 'traynor_snp_3ano',
                                                   'retorno_5_anos', 'traynor_cdi_5ano', 'traynor_ibov_5ano', 'traynor_snp_5ano',
                                                   'retorno_10_anos', 'traynor_cdi_10ano', 'traynor_ibov_10ano', 'traynor_snp_10ano'])
try:
    desempenho_df = calculated_cnpj.append(desempenho_df)
except:
    pass

desempenho_df.to_csv('desempenho.csv', index=False)

In [ ]:
# criando planilha de avg monthly results:

calculated_cnpj = pd.read_csv('taxas.csv')
calculated_cnpj_list = calculated_cnpj.CNPJ_FUNDO.tolist()
monthly_df = pd.read_csv('avg_monthly_return.csv')
monthly_list = monthly_df.CNPJ_FUNDO.tolist()

calculated_cnpj_list = [x for x in calculated_cnpj_list if x not in monthly_list]

avg_monthly_return_list = []
i=0
for cnpj in tqdm(calculated_cnpj_list):
    filename = f'{re.sub("[^0-9]", "", cnpj)}.csv'
    quotas = pd.read_csv(f'CNPJ desempenho/{filename}')
    quotas['DT_COMPTC'] = pd.to_datetime(quotas['DT_COMPTC'])
    
    montly_return = average_monthly_return(quotas)
    avg_monthly_return_list.append([cnpj, montly_return])

df = pd.DataFrame(avg_monthly_return_list, columns =['CNPJ_FUNDO', 'avg_monthly_return'])
 
monthly_df = monthly_df.append(df)
monthly_df.to_csv('avg_monthly_return.csv', index=False)
    


In [ ]:
# criando planilha de annualized return:

calculated_cnpj = pd.read_csv('taxas.csv')
calculated_cnpj_list = calculated_cnpj.CNPJ_FUNDO.tolist()
annualized_df = pd.read_csv('annualized_return.csv')
annualized_list = annualized_df.CNPJ_FUNDO.tolist()

calculated_cnpj_list = [x for x in calculated_cnpj_list if x not in annualized_list]

annualized_return_list = []
i=0
for cnpj in tqdm(calculated_cnpj_list):
    filename = f'{re.sub("[^0-9]", "", cnpj)}.csv'
    quotas = pd.read_csv(f'CNPJ desempenho/{filename}')
    quotas['DT_COMPTC'] = pd.to_datetime(quotas['DT_COMPTC'])
    
    annualized_return = annualized_return_function(quotas)
    annualized_return_list.append([cnpj, annualized_return])

df = pd.DataFrame(annualized_return_list, columns =['CNPJ_FUNDO', 'annualized_return'])
 
annualized_df = annualized_df.append(df)
annualized_df.to_csv('annualized_return.csv', index=False)
    


In [ ]:
# criando planilha de patrimonio_liquido:

calculated_cnpj = pd.read_csv('taxas.csv')
calculated_cnpj_list = calculated_cnpj.CNPJ_FUNDO.tolist()
patrimonio_df = pd.read_csv('patrimonio.csv')
patrimonio_list = patrimonio_df.CNPJ_FUNDO.tolist()

calculated_cnpj_list = [x for x in calculated_cnpj_list if x not in patrimonio_list]

patrimonio_list = []
i=0
for cnpj in tqdm(calculated_cnpj_list):
    filename = f'{re.sub("[^0-9]", "", cnpj)}.csv'
    quotas = pd.read_csv(f'CNPJ desempenho/{filename}')
    patrimonio = quotas['VL_PATRIM_LIQ'].iloc[-1]
    patrimonio_list.append([cnpj, patrimonio])

df = pd.DataFrame(patrimonio_list, columns =['CNPJ_FUNDO', 'patrimonio'])
 
patrimonio_df = patrimonio_df.append(df)
patrimonio_df.to_csv('patrimonio.csv', index=False)
    


In [ ]:
#criando planilha de max_daily_return

calculated_cnpj = pd.read_csv('desempenho.csv')
calculated_cnpj_list = calculated_cnpj.CNPJ_FUNDO.tolist()

max_daily_return_list = []
i=0
for cnpj in tqdm(calculated_cnpj_list):
    cnpj = '08.545.330/0001-74'
    filename = f'{re.sub("[^0-9]", "", cnpj)}.csv'
    quotas = pd.read_csv(f'CNPJ desempenho/{filename}')
    max_return, index = max_daily_return(quotas['VL_QUOTA'])
    max_daily_return_list.append([cnpj, max_return])

df = pd.DataFrame(max_daily_return_list, columns =['CNPJ_FUNDO', 'max_one_day_return'])
df.to_csv('max_one_day_return.csv', index=False)

In [ ]:
#criando planilha de n_cotistas

cotistas_cnpj = pd.read_csv('desempenho.csv')
cotistas_cnpj_list = cotistas_cnpj.CNPJ_FUNDO.tolist()

last_n_cotistas = []
i=0
for cnpj in tqdm(cotistas_cnpj_list):
    filename = f'{re.sub("[^0-9]", "", cnpj)}.csv'
    quotas = pd.read_csv(f'CNPJ desempenho/{filename}')
    cotistas = quotas['NR_COTST'].iloc[-1]
    last_n_cotistas.append([cnpj, cotistas])
    
df = pd.DataFrame(last_n_cotistas, columns =['CNPJ_FUNDO', 'n_cotistas'])
df.to_csv('n_cotistas.csv', index=False)

In [2]:
pd.set_option('display.max_rows', 10)
desempenho_df = pd.read_csv('desempenho.csv', index_col='CNPJ_FUNDO')

desempenho_df.dropna(subset = ["retorno_ytd"], inplace=True, axis=0)

fi_exclusivo = pd.read_csv('fi_exclusivo.csv', index_col='CNPJ_FUNDO')
desempenho_df = pd.merge(desempenho_df, fi_exclusivo, on='CNPJ_FUNDO', how='left')

unique_info = pd.read_csv('unique_fi_info.csv', index_col='CNPJ_FUNDO')
unique_info.drop(unique_info.columns.difference(['nomeAbreviado', 'investidorQualificado', 'idade', 'classe', 'situacao']), 1, inplace=True)
desempenho_df = pd.merge(desempenho_df, unique_info, on='CNPJ_FUNDO', how='left')

max_one_day_return_df = pd.read_csv('max_one_day_return.csv', index_col='CNPJ_FUNDO')

desempenho_df = pd.merge(desempenho_df, max_one_day_return_df, on='CNPJ_FUNDO', how='left')

n_cotistas_df = pd.read_csv('n_cotistas.csv', index_col='CNPJ_FUNDO')
desempenho_df = pd.merge(desempenho_df, n_cotistas_df, on='CNPJ_FUNDO', how='left')


desempenho_df.replace([np.inf, -np.inf], np.nan, inplace=True)
desempenho_df.dropna(subset = ["max_one_day_return"], inplace=True, axis=0)
desempenho_df.dropna(subset = ["retorno_3_anos"], inplace=True, axis=0)
desempenho_df = desempenho_df[desempenho_df.max_one_day_return < 2]
desempenho_df = desempenho_df[desempenho_df.investidorQualificado == False]
desempenho_df = desempenho_df[desempenho_df.fundo_exclusivo == False]
desempenho_df = desempenho_df[desempenho_df.n_cotistas > 100]
desempenho_df = desempenho_df[desempenho_df.situacao == 'EM FUNCIONAMENTO NORMAL']

tx_df = pd.read_csv('txadm_bench.csv', index_col='CNPJ_FUNDO')
desempenho_df = pd.merge(desempenho_df, tx_df['txa_adm_%aa'], on='CNPJ_FUNDO', how='left')

patrimonio_df = pd.read_csv('patrimonio.csv', index_col='CNPJ_FUNDO')
desempenho_df = pd.merge(desempenho_df, patrimonio_df['patrimonio'], on='CNPJ_FUNDO', how='left')

ann_return_df = pd.read_csv('annualized_return.csv', index_col='CNPJ_FUNDO')
desempenho_df = pd.merge(desempenho_df, ann_return_df['annualized_return'], on='CNPJ_FUNDO', how='left')


cols = ['nomeAbreviado', 'retorno_ytd', 'annualized_return', 'cumulative_returns', 'idade', 'annualized_sharpe', 
        'n_cotistas', 'classe', 'txa_adm_%aa', 'patrimonio',  'retorno_2020', 'retorno_1_mes', 'retorno_3_meses', 'retorno_1_ano', 'treynor_cdi_1ano', 
        'treynor_ibov_1ano', 'treynor_snp_1ano', 'retorno_3_anos', 'treynor_cdi_3ano', 'treynor_ibov_3ano', 
        'treynor_snp_3ano', 'retorno_5_anos', 'treynor_cdi_5ano', 'treynor_ibov_5ano', 'treynor_snp_5ano', 
        'retorno_10_anos', 'treynor_cdi_10ano', 'treynor_ibov_10ano', 'treynor_snp_10ano', 'fundo_exclusivo', 
        'investidorQualificado', 'max_one_day_return']
desempenho_df = desempenho_df[cols]
for index, row in desempenho_df.iterrows():
    if 'RF' in row['nomeAbreviado']:
        desempenho_df.at[index, 'classe'] = 'Fundo de Renda Fixa'
        
desempenho_df

,nomeAbreviado,retorno_ytd,annualized_return,cumulative_returns,idade,annualized_sharpe,n_cotistas,classe,txa_adm_%aa,patrimonio,...,treynor_cdi_5ano,treynor_ibov_5ano,treynor_snp_5ano,retorno_10_anos,treynor_cdi_10ano,treynor_ibov_10ano,treynor_snp_10ano,fundo_exclusivo,investidorQualificado,max_one_day_return
CNPJ_FUNDO,,,,,,,,,,,,,,,,,,,,,
00.068.305/0001-35,FIC FI CAIXA EMPREENDER RF LP,0.008131,0.077499,2.536832,10.0,31.082896,6909.0,Fundo de Renda Fixa,1.50,4.034465e+07,...,0.392826,342.995050,-1954.313948,0.952422,1.640302,2340.802464,-10716.707066,False,False,0.001385
00.071.477/0001-68,BB RF CURTO PRAZO AUTOMÁTICO EMPRESA FIC FI,0.005530,0.052281,1.371786,27.0,29.215796,292561.0,Fundo de Renda Fixa,1.75,3.397698e+10,...,0.299310,-10372.951665,-590.660576,0.607784,1.205233,-14503.684846,-2444.475881,False,False,0.000590
00.073.041/0001-08,BB BESC RF PRÁTICO CP FI,0.010813,0.095559,3.672744,9.0,5.140207,918.0,Fundo de Renda Fixa,1.00,1.688199e+07,...,0.449178,-3421.035565,-1173.415483,1.163654,1.901590,-7819.648691,-5110.829963,False,False,0.006505
00.089.915/0001-15,AMARIL FRANKLIN FIC FIM,0.006076,0.070202,2.154268,27.0,0.497046,132.0,Fundo Multimercado,1.50,2.683428e+07,...,0.416970,90.191095,-4048.689601,0.735729,1.589031,520.473667,-4501.758046,False,False,0.564265
00.180.995/0001-10,SAFRA EXECUTIVE MAX RF FIC FI,0.003858,0.097241,3.823899,10.0,14.916569,2144.0,Fundo de Renda Fixa,1.00,2.616016e+08,...,0.447653,20.155852,59.629766,1.221963,2.061959,106.892668,298.531918,False,False,0.003915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96.498.654/0001-66,BRADESCO FIA MULTI SETORIAL,0.018371,0.092793,2.364659,26.0,0.422813,490.0,Fundo de Ações,3.50,2.552894e+07,...,0.947505,0.948063,3.656780,0.807052,0.862089,0.914435,4.474525,False,False,0.123360
96.498.985/0001-04,BRADESCO FIC FIA IBOVESPA INDEXADO,0.056248,0.084907,1.716025,27.0,0.360432,680.0,Fundo de Ações,1.00,3.049542e+07,...,8.821340,0.942513,3.466524,0.566875,1.065832,0.566382,2.716211,False,False,0.145790
96.500.715/0001-82,BRADESCO FI RF REF DI ESTRELA,0.008414,0.089543,3.265180,11.0,19.362453,131.0,Fundo de Renda Fixa,1.00,2.664445e+08,...,0.418345,-19571.523260,-869.903495,1.052138,1.759295,-10968.190305,-3469.331464,False,False,0.015887


In [3]:
desempenho_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1571 entries, 00.068.305/0001-35 to 97.519.794/0001-36
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   nomeAbreviado          1571 non-null   object 
 1   retorno_ytd            1571 non-null   float64
 2   annualized_return      1571 non-null   float64
 3   cumulative_returns     1571 non-null   float64
 4   idade                  1571 non-null   float64
 5   annualized_sharpe      1571 non-null   float64
 6   n_cotistas             1571 non-null   float64
 7   classe                 1565 non-null   object 
 8   txa_adm_%aa            1571 non-null   float64
 9   patrimonio             1571 non-null   float64
 10  retorno_2020           1571 non-null   float64
 11  retorno_1_mes          1571 non-null   float64
 12  retorno_3_meses        1571 non-null   float64
 13  retorno_1_ano          1571 non-null   float64
 14  treynor_cdi_1ano       1571 no